# Ragas 和 LlamaIndex 示例

参考： https://docs.ragas.io/en/stable/howtos/integrations/llamaindex.html

结论：

- 可行
- 这里是自动生成测试集，测试的问题过长，可以考虑手动生成

In [1]:
# load the documents
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader("./data").load_data()

In [4]:
%%time

from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from llama_index.llms.openai_like import OpenAILike
from llama_index.embeddings.ollama import OllamaEmbedding

# generator with openai models
generator_llm = OpenAILike(
    model="qwen2", 
    api_base="http://ape:3000/v1", 
    api_key="sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750",
    is_chat_model=True,
    temperature=0.1,
    request_timeout=60.0
)
critic_llm = OpenAILike(
    model="qwen2", 
    api_base="http://ape:3000/v1", 
    api_key="sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750",
    is_chat_model=True,
    temperature=0.1,
    request_timeout=60.0
)
embeddings = OllamaEmbedding(
    model_name="quentinz/bge-large-zh-v1.5",
    base_url="http://monkey:11434",
    ollama_additional_kwargs={"mirostat": 0}, # -mirostat N 使用 Mirostat 采样。
)

generator = TestsetGenerator.from_llama_index(
    generator_llm=generator_llm,
    critic_llm=critic_llm,
    embeddings=embeddings,
)

CPU times: user 211 µs, sys: 22 µs, total: 233 µs
Wall time: 237 µs


In [5]:
%%time

# generate testset
testset = generator.generate_with_llamaindex_docs(
    documents,
    test_size=5,
    distributions={simple: 0.5, reasoning: 0.25, multi_context: 0.25},
)

embedding nodes:   0%|          | 0/22 [00:00<?, ?it/s]

Filename and doc_id are the same for all nodes.


Generating:   0%|          | 0/5 [00:00<?, ?it/s]

max retries exceeded for ReasoningEvolution(generator_llm=LlamaIndexLLMWrapper(run_config=RunConfig(timeout=60, max_retries=15, max_wait=90, max_workers=16, thread_timeout=80.0, exception_types=(<class 'Exception'>,), log_tenacity=False)), docstore=InMemoryDocumentStore(splitter=<langchain_text_splitters.base.TokenTextSplitter object at 0x7fd6af8ab250>, nodes=[Node(page_content='收藏\n查看我的收藏\n0有用+1\n北海公园（Beihai Park），位于北京市西城区文津街1号，东邻景山公园，南濒中南海，北连什刹海，全园占地68.2万平方米（其中水域面积38.9万平方米，陆地面积29.3万平方米），1925年开放为公园。 [3] [28]\n北海及团城于1961年被中华人民共和国中央人民政府公布为第一批全国重点文物保护单位。 [32]北海公园也是国家AAAA级旅游景区。 [33]\n2023年4月，北海公园入选第二批国家级文明旅游示范单位公示名单。 [37]\n- 中文名\n- 北海公园\n- 外文名\n- Beihai Park\n- 地理位置\n- 北京市西城区文津街1号\n- 气候条件\n- 温带季风气候\n- 开放时间\n- 4月1日-10月31日 06:00-21:00；11月1日-次年3月31日 06:30-20:00 [1]\n- 景点级别\n- AAAA级\n- 门票价格\n- 旺季10元，淡季5元\n- 占地面积\n- 682000 m²（其中水面积38.9万平方米，陆地面积29.3万平方米）\n- 著名景点\n- 琼华岛、永安寺、北海白塔\n北海公园原为辽燕京城东北一片湖泊，名金海。中有小岛名瑶屿，辽在此建“瑶屿行宫”。金大定三年至十九年（1163年—1179年），金世宗仿照北宋汴梁（今河南开封）艮岳园，建琼华岛，并从“艮岳”御苑运来大量太湖石砌成假山岩洞，在中都的东北郊

CPU times: user 1.2 s, sys: 54.6 ms, total: 1.25 s
Wall time: 2min 37s


In [27]:
df = testset.to_pandas()
df.head()

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,在这段关于北海公园的历史和文化活动描述中，提到了《三希堂法帖》石刻收集了中国从魏晋至明末的1...,[�等奉命编次的魏晋以来历代名墨迹的钩摹石刻，共495方，清代帝后常来此阅赏古代翰墨。楼内四...,《三希堂法帖》中包含了中国从魏晋至明末的135位著名书法家的作品,simple,[{'file_path': '/root/notebook/my-jupyter-note...,True
1,小西天的历史背景和主要建筑特色是什么？,[�什刹海，属于中国古代皇家园林。 [1]全园以北海为中心，全园占地69公顷（其中水面39公...,小西天始建于清乾隆三十三年（1768年），建成于清乾隆三十五年（1770年），是为清乾隆皇帝...,simple,[{'file_path': '/root/notebook/my-jupyter-note...,True
2,万岁山在元清两朝经历了哪些扩建与修缮？明朝朱棣迁都后及宣宗时期，它如何从琼华岛发展为广寒殿并...,[�建大宁离宫（亦称太宁宫）。 [4]从那时起，北海就基本形成了今天皇家宫苑格局。当时把挖“...,万岁山在元朝时期经历了三次扩建琼华岛，重建广寒殿。广寒殿作为帝王朝会之处，规模宏大，内部装饰...,multi_context,[{'file_path': '/root/notebook/my-jupyter-note...,True
3,在北海公园的发展历程中，'锡殿'或'太素殿'如何成为突出的建筑，并在其各朝代的建设与改动中发...,[（现五龙亭处）建“太素殿”，由于用锡做材料，又称为“锡殿”，也叫“避暑凉殿”。修建此殿役使...,"在北海公园的发展历程中，""锡殿""或""太素殿""因其使用锡材料而得名，也被称为""避暑凉殿""。这座...",multi_context,[{'file_path': '/root/notebook/my-jupyter-note...,True


In [8]:
%%time

# build query engine
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.settings import Settings

Settings.llm=OpenAILike(
    model="qwen2", 
    api_base="http://ape:3000/v1", 
    api_key="sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750",
    is_chat_model=True,
    temperature=0.1,
    request_timeout=60.0
)

Settings.embed_model=OllamaEmbedding(
    model_name="quentinz/bge-large-zh-v1.5",
    base_url="http://monkey:11434",
    ollama_additional_kwargs={"mirostat": 0}, # -mirostat N 使用 Mirostat 采样。
)

vector_index = VectorStoreIndex.from_documents(documents)

query_engine = vector_index.as_query_engine()

CPU times: user 74.9 ms, sys: 599 µs, total: 75.5 ms
Wall time: 1.59 s


In [14]:
# convert it to pandas dataset
df = testset.to_pandas()
df["question"][1]

'小西天的历史背景和主要建筑特色是什么？'

In [15]:
%%time

query_engine = vector_index.as_query_engine(top_k=5)
response_vector = query_engine.query(df["question"][1])

print(response_vector)

小西天始建于清乾隆三十三年（1768年），建成于清乾隆三十五年（1770年）。它最初是为清朝乾隆皇帝的母亲孝圣皇太后祝寿祈福而建的。主体建筑极乐世界总面积达1200平方米，横梁跨度达到13.5米，是中国最大的方亭式宫殿建筑。殿内装饰精细，四面窗扉和楠木扇上细刻花纹，高处悬挂金匾，上书“极乐世界”，为乾隆御笔。殿上方有金光灿灿的八角穹窿团龙藻井。极乐世界四周环水，并设有桥连接，东西南北各有琉璃牌坊一座，四角各有一座小方亭。正南面有一道月牙河，上架雕栏石桥。殿内原有南海普陀泥塑一座，山上布有226尊罗汉佛像，山下绘满海水，象征佛界普陀胜境，因此得名“罗汉山”和“海岛”。
CPU times: user 14 ms, sys: 510 µs, total: 14.5 ms
Wall time: 5.22 s


In [16]:
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
)
from ragas.metrics.critique import harmfulness

metrics = [
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
    harmfulness,
]

In [17]:
%%time

# using GPT 3.5, use GPT 4 / 4-turbo for better accuracy
evaluator_llm = OpenAILike(
    model="gpt-3.5-turbo", 
    api_base="http://ape:3000/v1", 
    api_key="sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750",
    is_chat_model=True,
    temperature=0.1,
    request_timeout=60.0
)

CPU times: user 99 µs, sys: 9 µs, total: 108 µs
Wall time: 111 µs


In [18]:
# convert to HF dataset
ds = testset.to_dataset()

ds_dict = ds.to_dict()
ds_dict["question"]
ds_dict["ground_truth"]

['《三希堂法帖》中包含了中国从魏晋至明末的135位著名书法家的作品',
 '小西天始建于清乾隆三十三年（1768年），建成于清乾隆三十五年（1770年），是为清乾隆皇帝的母亲孝圣皇太后祝寿祈福而建。主体建筑极乐世界总面积达1200平方米，是中国最大的方亭式宫殿建筑，横梁跨度13.5米。殿内有金匾上书“极乐世界”，为乾隆御笔，并有八角穹窿团龙藻井。殿四面环水，有桥可通，东西南北各有琉璃牌坊一座，四角各有一座小方亭。殿内原有南海普陀泥塑和226尊罗汉佛像，以象征佛界普陀胜境，故有“罗汉山”和“海岛”之称。',
 '万岁山在元朝时期经历了三次扩建琼华岛，重建广寒殿。广寒殿作为帝王朝会之处，规模宏大，内部装饰豪华，包括放置渎山大玉海、建玉殿放五山珍玉榻等。至元八年（1271年），琼华岛改称万寿山。明朝朱棣迁都北京后，在元朝的基础上对北海进行扩充和修葺，保持了元代北海的格局。宣宗朱瞻基在明代宣德年间对万岁山进行了大规模扩建和修缮，在圆坻修复仪天殿、建犀山抬圆殿，并将团城东部拆桥填土与陆地相连。顺治八年（1651年），为民族和睦，清世祖福临在广寒殿废址上建藏式白塔，并改山名为白塔山。乾隆六年至三十六年（1741年—1771年）期间，对北海进行了大规模修葺和增建，引入江南园林的精华、文人写意、山水园林元素，先后建成静心斋、画舫斋、濠濮间等‘园中之园’。',
 '在北海公园的发展历程中，"锡殿"或"太素殿"因其使用锡材料而得名，也被称为"避暑凉殿"。这座建筑在各朝代的建设与改动中发挥了重要作用。明朝万历七年（1579年），"万岁山"上的广寒殿坍毁后，北海公园经历了多次扩建和修缮。清顺治八年（1651年），为了民族和睦，在广寒殿废址上建起了藏式白塔，并在塔前建造了"白塔寺"（正觉殿为山门）。这不仅丰富了北海公园的文化内涵，也体现了清朝对多元文化的包容。此外，清乾隆年间（1741年至1771年），北海进行了大规模的修葺和增建，期间建起了许多亭、台、殿、阁，如静心斋、画舫斋等"园中之园"，进一步提升了北海公园的艺术价值和皇家宫苑格局。这些建设与改动不仅保留了北海的历史风貌，也融入了江南园林的精华，展现了文人写意和山水园林的魅力。']

In [20]:
%%time

from ragas.integrations.llama_index import evaluate

result = evaluate(
    query_engine=query_engine,
    metrics=metrics,
    dataset=ds_dict,
    llm=evaluator_llm,
    embeddings=Settings.embed_model,
)

Running Query Engine:   0%|          | 0/4 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/20 [00:00<?, ?it/s]

n values greater than 1 not support for LlamaIndex LLMs
n values greater than 1 not support for LlamaIndex LLMs
n values greater than 1 not support for LlamaIndex LLMs
n values greater than 1 not support for LlamaIndex LLMs


CPU times: user 495 ms, sys: 23.1 ms, total: 518 ms
Wall time: 1min 17s


In [21]:
result

{'faithfulness': 0.4875, 'answer_relevancy': 0.4777, 'context_precision': 1.0000, 'context_recall': 0.9286, 'harmfulness': 0.5000}

In [22]:
result.to_pandas()

,question,contexts,answer,ground_truth,faithfulness,answer_relevancy,context_precision,context_recall,harmfulness
0,在这段关于北海公园的历史和文化活动描述中，提到了《三希堂法帖》石刻收集了中国从魏晋至明末的1...,[清高宗弘历乾隆六年至三十六年（1741年—1771年），对北海进行了大规模的修葺和增建，前...,根据提供的上下文信息，并没有提及《三希堂法帖》或其包含的书法家作品时期的具体内容。因此，无法...,《三希堂法帖》中包含了中国从魏晋至明末的135位著名书法家的作品,0.00,0.000000,1.0,1.000000,1
1,小西天的历史背景和主要建筑特色是什么？,[广寒殿东西宽120尺，深62尺，高50尺，殿广7间，作为帝王朝会之处。殿中放置“渎山大玉海...,小西天始建于清乾隆三十三年（1768年），建成于清乾隆三十五年（1770年）。它最初是为清朝...,小西天始建于清乾隆三十三年（1768年），建成于清乾隆三十五年（1770年），是为清乾隆皇帝...,1.00,0.607257,1.0,1.000000,1
2,万岁山在元清两朝经历了哪些扩建与修缮？明朝朱棣迁都后及宣宗时期，它如何从琼华岛发展为广寒殿并...,[收藏\n查看我的收藏\n0有用+1\n北海公园（Beihai Park），位于北京市西城区...,万岁山在元朝时期经历了三次扩建与修缮。最初，金世宗仿照北宋汴梁的艮岳园，在北海公园的东北部修...,万岁山在元朝时期经历了三次扩建琼华岛，重建广寒殿。广寒殿作为帝王朝会之处，规模宏大，内部装饰...,0.00,0.611885,1.0,1.000000,0
3,在北海公园的发展历程中，'锡殿'或'太素殿'如何成为突出的建筑，并在其各朝代的建设与改动中发...,[广寒殿东西宽120尺，深62尺，高50尺，殿广7间，作为帝王朝会之处。殿中放置“渎山大玉海...,在北海公园的发展历程中，“锡殿”（又称“避暑凉殿”）和“太素殿”的建设与改动体现了其作为皇家...,"在北海公园的发展历程中，""锡殿""或""太素殿""因其使用锡材料而得名，也被称为""避暑凉殿""。这座...",0.95,0.691624,1.0,0.714286,0
